<a href="https://colab.research.google.com/github/1kaiser/ICESAT2/blob/main/GraphConnectionICESAT2Photon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# prerequisite
![image](https://user-images.githubusercontent.com/26379748/236386635-884d6a02-a953-4c4b-8e97-7c2cdb2a2346.png)
![image](https://user-images.githubusercontent.com/26379748/236386689-56fc5213-49c3-46ab-b167-2e9ee92c4523.png)
![image](https://user-images.githubusercontent.com/26379748/236386810-79ca2496-9c6a-4484-aefb-adcfb23f74d2.png)
![image](https://user-images.githubusercontent.com/26379748/236387012-e1b05c27-02de-4671-8e66-27f7629e99e9.png)


In [ ]:
list_A = ['https://n5eil02u.ecs.nsidc.org/esir/5000004058045/239104701/processed_ATL03_20211108045334_07141302_005_01.h5',
 'https://n5eil02u.ecs.nsidc.org/esir/5000004058045/226925046/processed_ATL03_20210627231818_00661206_005_01.h5',
 'https://n5eil02u.ecs.nsidc.org/esir/5000004058045/226819567/processed_ATL03_20210530004215_10111106_005_01.h5',
 'https://n5eil02u.ecs.nsidc.org/esir/5000004058045/226455818/processed_ATL03_20210208175352_07141002_005_01.h5',
 'https://n5eil02u.ecs.nsidc.org/esir/5000004058045/225517656/processed_ATL03_20200211111433_07140602_005_01.h5',
 'https://n5eil02u.ecs.nsidc.org/esir/5000004074543/224813989/processed_ATL03_20190427130425_04470306_005_01.h5',
 'https://n5eil02u.ecs.nsidc.org/esir/5000004074543/223502286/processed_ATL03_20190126172441_04470206_005_01.h5',
]

list_B = [
    'https://n5eil02u.ecs.nsidc.org/esir/5000004077220/223754368/processed_ATL03_20190213043533_07140202_005_01.h5',
    'https://n5eil02u.ecs.nsidc.org/esir/5000004077220/224821719/processed_ATL03_20190515001514_07140302_005_01.h5',
    'https://n5eil02u.ecs.nsidc.org/esir/5000004077220/225517656/processed_ATL03_20200211111433_07140602_005_01.h5',
    'https://n5eil02u.ecs.nsidc.org/esir/5000004077319/225806241/processed_ATL03_20200531180255_10110706_005_01.h5',
    'https://n5eil02u.ecs.nsidc.org/esir/5000004077220/226455818/processed_ATL03_20210208175352_07141002_005_01.h5',
    'https://n5eil02u.ecs.nsidc.org/esir/5000004077319/226819567/processed_ATL03_20210530004215_10111106_005_01.h5',
    'https://n5eil02u.ecs.nsidc.org/esir/5000004077220/239104701/processed_ATL03_20211108045334_07141302_005_01.h5',
]

for i in list_B:
  !wget --http-user=kroy0001 --http-password=/#j%kWrPA,8.HRe {i}


In [ ]:
!pip install rasterio

In [ ]:
!wget https://github.com/1kaiser/ICESAT2/releases/download/1/ASTER_DEM.tif #getting the dem
!wget https://github.com/1kaiser/ICESAT2/releases/download/1/pongdam_dem.tif #getting Pong Dam dem
!wget https://github.com/1kaiser/ICESAT2/releases/download/1/landsat2.zip
!unzip '*.zip'
!rm -r *.zip

# **sat map visualization**

In [ ]:
image_dir = r'/content/landsat2'

#############################################################################
prefix = ""
bandend = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B09", "B10", "B11", "doy", "qc_500m", "raz", "state_500m", "szen", "vzen"]
DayOY = "_doy\[0-9]+_aid0001"
fileExt = r'.tif'
expression_b1 = prefix+bandend[0]

expression_b2 = prefix+bandend[1]

expression_b3 = prefix+bandend[2]
expression_b4 = prefix+bandend[3]
expression_b5 = prefix+bandend[4]
expression_b6 = prefix+bandend[5]
expression_b7 = prefix+bandend[6]
expression_b9 = prefix+bandend[7]
expression_b10 = prefix+bandend[8]
expression_b11 = prefix+bandend[9]

temp_dir = r'/content/'
import os

imgs_list_b2 = [f for f in os.listdir(image_dir) if f.__contains__(expression_b2)]

imgs_list_b2.sort(reverse=True) 

                    #<<<< to start file streaming from the last date 2022 >> 2021 >> 2020 ....

imgs_path_b2 = [os.path.join(image_dir, i) for i in imgs_list_b2 if i != 'outputs']
print(len(imgs_path_b2))

In [ ]:
!pip install rasterio

In [ ]:
import os
import glob
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from rasterio.plot import show_hist
from rasterio.warp import reproject, Resampling

# Find all tif files in the folder

tif_files = imgs_path_b2
dem_path = '/content/DEM.tif'

# # Define the function to calculate NDSI
def ndsi1(A, B, C):
    green = A.astype(float)
    swir = B.astype(float)
    water = C.astype(float)
    return ((green - swir) / (green + swir)) 
    
# Loop over the tif files and calculate NDSI for each one
for i, file in enumerate(tif_files):
    # Open the tif file with rasterio
    src = rasterio.open(file.replace(expression_b2, expression_b2))
    B2 = src.read()
    B4 = rasterio.open(file.replace(expression_b2, expression_b4)).read()
    B6 = rasterio.open(file.replace(expression_b2, expression_b6)).read()
    B1 = rasterio.open(file.replace(expression_b2, expression_b1)).read()
    B3 = rasterio.open(file.replace(expression_b2, expression_b3)).read()
    B5 = rasterio.open(file.replace(expression_b2, expression_b5)).read()
    B7 = rasterio.open(file.replace(expression_b2, expression_b7)).read()
    B9 = rasterio.open(file.replace(expression_b2, expression_b9)).read()
    B10 = rasterio.open(file.replace(expression_b2, expression_b10)).read()
    B11 = rasterio.open(file.replace(expression_b2, expression_b11)).read()

    # # Calculate NDSI
    # ndsi_array = ndsi1(B3, B6, B2)
    # # Save the NDSI file
    # with rasterio.open('ndsi_output.tif', 'w', driver='GTiff', width=src.width, height=src.height, count=1, dtype=rasterio.float32, crs=src.crs, transform=src.transform) as dst:
    #   ndsi_array = ndsi_array[0,:,:]
    #   dst.write(ndsi_array, 1)

    # # Open the DEM raster
    # with rasterio.open(dem_path) as dem:
    #     dem_profile = dem.profile
    #     dem_crs = dem.crs
    #     dem_transform = dem.transform
    #     dem_width = dem.width
    #     dem_height = dem.height

    # # Reproject the NDSI raster to the same resolution as the DEM
    # with rasterio.open('/content/ndsi_output.tif') as ndsi_src:
    #     ndsi_profile, ndsi_array = reproject_raster(
    #         src=ndsi_src,
    #         dst_crs=dem_crs,
    #         dst_transform=dem_transform,
    #         dst_width=dem_width,
    #         dst_height=dem_height)

    # # Save the NDSI raster
    # with rasterio.open('ndsi_output_reprojected.tif', 'w', **ndsi_profile) as ndsi_dst:
    #     ndsi_array = ndsi_array[0,:,:]
    #     ndsi_dst.write(ndsi_array.astype(rasterio.float32), 1)

    B1 = B1.astype(float)/(2^16-1)/10;print(B1.max())    
    B2 = B2.astype(float)/(2^16-1)/10;print(B2.max())
    B3 = B3.astype(float)/(2^16-1)/10;print(B3.max())
    B4 = B4.astype(float)/(2^16-1)/10;print(B4.max())
    B5 = B5.astype(float)/(2^16-1)/10;print(B5.max())
    B6 = B6.astype(float)/(2^16-1)/10;print(B6.max())
    B7 = B7.astype(float)/(2^16-1)/10;print(B7.max())
    B9 = B9.astype(float)/(2^16-1)/10;print(B9.max())
    B10 = B10.astype(float)/(2^16-1)/10;print(B10.max())
    B11 = B11.astype(float)/(2^16-1)/10;print(B11.max())
    # ndsi_path = '/content/ndsi_output_reprojected.tif'
    # NDSI = rasterio.open(ndsi_path).read().astype(float)
    # DEM = rasterio.open(dem_path).read().astype(float)
    # slope_path = '/content/SLOPE.tif'
    # aspect_path = '/content/ASPECT.tif'
    # SLOPE = rasterio.open(slope_path).read().astype(float)
    # ASPECT = rasterio.open(aspect_path).read().astype(float)
    ndsi_array = ndsi1(B4, B6, B2)
    # Create a figure
    # Create a 2x4 grid of subplots
    fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(15, 5))

    # Plot the data in each subplot
    axs[0, 0].scatter( ndsi_array[B1 > 0], B1[B1 > 0], s=1, c='b', alpha=0.1); axs[0, 0].set_xlabel('NDSI');axs[0,0].set_ylabel('B1');axs[0,0].grid(True);axs[0,0].set_xlim(-1, 1);axs[0,0].set_ylim(0, 100)
    axs[0, 1].scatter( ndsi_array[B2 > 0], B2[B2 > 0], s=1, c='b', alpha=0.1); axs[0, 1].set_xlabel('NDSI');axs[0,1].set_ylabel('B2');axs[0,1].grid(True);axs[0,1].set_xlim(-1, 1);axs[0,1].set_ylim(0, 100)
    axs[0, 2].scatter( ndsi_array[B3 > 0], B3[B3 > 0], s=1, c='b', alpha=0.1); axs[0, 2].set_xlabel('NDSI');axs[0,2].set_ylabel('B3');axs[0,2].grid(True);axs[0,2].set_xlim(-1, 1);axs[0,2].set_ylim(0, 100)
    axs[0, 3].scatter( ndsi_array[B4 > 0], B4[B4 > 0], s=1, c='b', alpha=0.1); axs[0, 3].set_xlabel('NDSI');axs[0,3].set_ylabel('B4');axs[0,3].grid(True);axs[0,3].set_xlim(-1, 1);axs[0,3].set_ylim(0, 100)
    axs[0, 4].scatter( ndsi_array[B5 > 0], B5[B5 > 0], s=1, c='b', alpha=0.1); axs[0, 4].set_xlabel('NDSI');axs[0,4].set_ylabel('B5');axs[0,4].grid(True);axs[0,4].set_xlim(-1, 1);axs[0,4].set_ylim(0, 100)
    axs[1, 0].scatter( ndsi_array[B6 > 0], B6[B6 > 0], s=1, c='b', alpha=0.1); axs[1, 0].set_xlabel('NDSI');axs[1,0].set_ylabel('B6');axs[1,0].grid(True);axs[1,0].set_xlim(-1, 1);axs[1,0].set_ylim(0, 100)
    axs[1, 1].scatter( ndsi_array[B7 > 0], B7[B7 > 0], s=1, c='b', alpha=0.1); axs[1, 1].set_xlabel('NDSI');axs[1,1].set_ylabel('B7');axs[1,1].grid(True);axs[1,1].set_xlim(-1, 1);axs[1,1].set_ylim(0, 100)
    axs[1, 2].scatter( ndsi_array[B9 > 0], B9[B9 > 0], s=1, c='b', alpha=0.1); axs[1, 2].set_xlabel('NDSI');axs[1,2].set_ylabel('B9');axs[1,2].grid(True);axs[1,2].set_xlim(-1, 1);axs[1,2].set_ylim(0, 100)
    axs[1, 3].scatter( ndsi_array[B10 > 0], B10[B10 > 0], s=1, c='b', alpha=0.1); axs[1, 3].set_xlabel('NDSI');axs[1,3].set_ylabel('B10');axs[1,3].grid(True);axs[1,3].set_xlim(-1, 1);axs[1,3].set_ylim(0, 100)
    axs[1, 4].scatter( ndsi_array[B11 > 0], B11[B11 > 0], s=1, c='b', alpha=0.1); axs[1, 4].set_xlabel('NDSI');axs[1,4].set_ylabel('B11');axs[1,4].grid(True);axs[1,4].set_xlim(-1, 1);axs[1,4].set_ylim(0, 100)
    
    # axs[1, 2].scatter( NDSI[DEM > 0], DEM[DEM > 0], s=1, c='r', alpha=0.05); axs[1, 2].set_xlabel('NDSI');axs[1,2].set_ylabel('DEM');axs[1,2].grid(True);axs[1,2].set_xlim(-1, 1)
    # axs[1, 3].scatter( NDSI[SLOPE > 0], SLOPE[SLOPE > 0], s=1, c='g', alpha=0.05); axs[1, 3].set_xlabel('NDSI');axs[1,3].set_ylabel('SLOPE');axs[1,3].grid(True);axs[1,3].set_xlim(-1, 1)
    # axs[1, 4].scatter( NDSI[ASPECT > 0], ASPECT[ASPECT > 0], s=1, c='y', alpha=0.05); axs[1, 4].set_xlabel('NDSI');axs[1,4].set_ylabel('ASPECT');axs[1,4].grid(True);axs[1,4].set_xlim(-1, 1)

    # axs[1, 3].remove()
    # Add a title to the plot
    fig.suptitle('NDSI vs Bands')
    # adjust the spacing between subplots
    fig.subplots_adjust(hspace=0.4, wspace=0.4)

    # ax.legend([ 'Non-water', 'Water'])
    plt.show()

def reproject_raster(src, dst_crs, dst_transform, dst_width, dst_height):
    """
    Reproject raster to the same resolution as the DEM.
    """
    dst_profile = src.profile.copy()
    dst_profile.update({
        'crs': dst_crs,
        'transform': dst_transform,
        'width': dst_width,
        'height': dst_height
    })
    dst_array = np.zeros((src.count, dst_height, dst_width), dtype=src.dtypes[0])
    reproject(
        source=rasterio.band(src, 1),
        destination=dst_array,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=dst_transform,
        dst_crs=dst_crs,
        resampling=Resampling.nearest)
    return dst_profile, dst_array


# stage 1

In [ ]:
#@title **plotting of the map bands as rgb** 
import rasterio
import matplotlib.pyplot as plt
import numpy as np


def plot_rgb_raster(r1, r2, r3, axes):
    raster1, raster2, raster3 = rasterio.open(r1), rasterio.open(r2), rasterio.open(r3)    # Open the raster file
    r, g, b = raster1.read(1), raster2.read(1), raster3.read(1)    # Read the RGB bands
    rgb = np.dstack((r, g, b))    # Stack the bands into an RGB image
    axes.imshow(rgb)    # Plot the RGB image
    return fig
    
r1 = '/content/landsat2/HLSL30.020_B01_doy2019109_aid0001_43N.tif'
r2 = '/content/landsat2/HLSL30.020_B02_doy2019109_aid0001_43N.tif'
r3 = '/content/landsat2/HLSL30.020_B03_doy2019109_aid0001_43N.tif'
r4 = '/content/landsat2/HLSL30.020_B01_doy2019109_aid0001_43N.tif'
r5 = '/content/landsat2/HLSL30.020_B02_doy2019109_aid0001_43N.tif'
r6 = '/content/landsat2/HLSL30.020_B03_doy2019109_aid0001_43N.tif'
r7 = '/content/landsat2/HLSL30.020_B01_doy2019109_aid0001_43N.tif'
r9 = '/content/landsat2/HLSL30.020_B02_doy2019109_aid0001_43N.tif'
r10 = '/content/landsat2/HLSL30.020_B03_doy2019109_aid0001_43N.tif'
r11 = '/content/landsat2/HLSL30.020_B01_doy2019109_aid0001_43N.tif'

fig, axs = plt.subplots(1, 10, figsize=(60,20))

fig = plot_rgb_raster(r1, r2, r3, axs[0])
fig = plot_rgb_raster(r2, r3, r4, axs[1])
fig = plot_rgb_raster(r3, r4, r5, axs[2])
fig = plot_rgb_raster(r4, r5, r6, axs[3])
fig = plot_rgb_raster(r5, r6, r7, axs[4])
fig = plot_rgb_raster(r6, r7, r9, axs[5])
fig = plot_rgb_raster(r7, r9, r10, axs[6])
fig = plot_rgb_raster(r9, r10, r11, axs[7])
fig = plot_rgb_raster(r10, r11, r1, axs[8])
fig = plot_rgb_raster(r11, r1, r2, axs[9])
plt.show()  # Show the plot

In [ ]:
#@title **Elevation & Distance Sample Extraction** { vertical-output: true }
filename = "/content/processed_ATL03_20190213043533_07140202_005_01.h5" #@param {type:"string"}
track_name = "gt2r" #@param {type:"string"}
start_point = 1000 #@param {type:"integer"}
distance_range = 1500 #@param {type:"integer"}
elevation_band_thickness = 150 #@param {type:"integer"}


import h5py
import numpy as np
import matplotlib.pyplot as plt

def create_geometric_graph(elevations, distances, elevation_weight, distance_weight):
    # Create a complete graph with nodes representing the given points
    num_points = len(elevations)
    edges = [(i, j) for i in range(num_points) for j in range(i+1, num_points)]
    
    # Compute the weights of the edges between all pairs of nodes
    weights = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j in range(i+1, num_points):
            elevation_diff = abs(elevations[i] - elevations[j])
            weights[i][j] = elevation_weight * elevation_diff + distance_weight * abs(distances[i] - distances[j])
            weights[j][i] = weights[i][j]
    
    # Create a GeometricGraph object from the edge list and weights
    geom_graph = (edges, weights)
    
    return geom_graph

def extract_elevation_data(filename, track_name, start_point, distance_range, elevation_band_thickness):
    # Open the HDF5 file and extract data
    f = h5py.File(filename, 'r')
    h_ph_data = f['/'+str(track_name)+'/heights/h_ph']
    lat_ph_data = f['/'+str(track_name)+'/heights/lat_ph']
    lon_ph_data = f['/'+str(track_name)+'/heights/lon_ph']

    # Calculate distance along track using haversine formula
    def haversine(lat1, lon1, lat2, lon2):
        R = 6371.0  # Earth radius in km
        dLat = np.radians(lat2 - lat1)
        dLon = np.radians(lon2 - lon1)
        lat1 = np.radians(lat1)
        lat2 = np.radians(lat2)
        a = np.sin(dLat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dLon/2)**2
        c = 2*np.arcsin(np.sqrt(a))
        return R*c*1000.0

    dist_ph_along_data = [haversine(lat_ph_data[0], lon_ph_data[0], lat_ph_data[i], lon_ph_data[i]) for i in range(len(lat_ph_data))]

    # Select data based on a starting point, distance range, and elevation band thickness
    dist_ph_along_data = np.array(dist_ph_along_data)
    h_ph_data = np.array(h_ph_data)

    c = np.where((dist_ph_along_data > start_point) & (dist_ph_along_data < start_point + distance_range))[0]

    median_height = np.median(h_ph_data[c])
    d = np.where((dist_ph_along_data > start_point) & (dist_ph_along_data < start_point + distance_range) & 
                 (h_ph_data > median_height - elevation_band_thickness/2) &
                 (h_ph_data < median_height + elevation_band_thickness/2))[0]

    # Extract elevation and distance data
    elevation = h_ph_data[d]
    distance = dist_ph_along_data[d]

    return distance, elevation

# filename = '/content/processed_ATL03_20200211111433_07140602_005_01.h5'
# track_name = 'gt2r'
# start_point = 25500
# distance_range = 300
# elevation_band_thickness = 50

distance, elevation = extract_elevation_data(filename, track_name, start_point, distance_range, elevation_band_thickness)


## test visualization

In [ ]:
import matplotlib.pyplot as plt
import jax.numpy as jnp


# Generate the elevation and distance values.
elevation = elevation
distance = distance


# Create a points array.
def create_points(elevation, distance):
  return jnp.array([
    [elevation[i], distance[i]]
    for i in range(len(elevation))
  ])


import numpy as np

def create_adjacency_matrix(points, threshold):
  distance_matrix = np.linalg.norm(points[:, None] - points, axis=2)
  mask = distance_matrix < threshold
  adjacency_matrix = np.where(mask, 1, 0)
  return adjacency_matrix


# Create the points array.
points = create_points(elevation, distance)
n = len(elevation)
adjacency_matrix = np.zeros((n, n))
adjacency_matrix = create_adjacency_matrix(points, 3)
# # Create an adjacency matrix.
# adjacency_matrix = np.zeros((n, n))
# for i in range(n):
#   for j in range(n):
#     if np.linalg.norm(points[i] - points[j]) < 3:
#       adjacency_matrix[i, j] = 1

# Plot the points and the connections.
plt.figure()
plt.scatter(points[:, 1], points[:, 0], s = 1)

for i in range(n):
  for j in range(n):
    if adjacency_matrix[i, j] == 1:
      plt.plot( [points[i, 1], points[j, 1]], [points[i, 0], points[j, 0]], 'r-')

plt.xlabel("X")
plt.ylabel("Y")
plt.show()


# test stage

In [ ]:
#@title test code
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt


# Create a points array.
def create_points(elevation, distance):
  return jnp.array([
    [elevation[i], distance[i]]
    for i in range(len(elevation))
  ])

# Swap the axes of the points array.
def swap_axes(points):
  return points[:, ::-1]

# Create a distance matrix between the points.
def create_distance_matrix(points):
  return jnp.linalg.norm(points[:, None] - points, axis=2)

# Create a mask of the points that are within the threshold distance.
def create_mask(distance_matrix, threshold):
  return jnp.less_equal(distance_matrix, threshold)

# Create an adjacency matrix from the mask.
def create_adjacency_matrix(mask):
  return jnp.where(mask, 1, 0)

# Convert the adjacency matrix to a NumPy array of size 20x20.
def convert_adjacency_matrix_to_numpy(adjacency_matrix, n):
  return adjacency_matrix.reshape((n, n))






# Plot the points and the connections using the adjacency matrix.
def plot_points_and_connections(points, adjacency_matrix, xlabel, ylabel):
  plt.figure()
  plt.scatter(points[:, 1], points[:, 0], s = 1)
  for i in range(len(adjacency_matrix)):
    for j in range(len(adjacency_matrix[0])):
      if adjacency_matrix[i][j] == 1:
        plt.plot([points[i, 1], points[j, 1]], [points[i, 0], points[j, 0]], 'r-')

  plt.xlabel(xlabel)
  plt.ylabel(ylabel)

  plt.show()

# Get the number of points from the user.
# n = int(input("Enter the number of points: "))
n = len(elevation)
# Generate the elevation and distance values.
elevation = elevation
distance = distance

# Create the points array.
points = create_points(elevation, distance)

# Create the distance matrix between the points.
distance_matrix = create_distance_matrix(points)

# Create a mask of the points that are within the threshold distance.
threshold = 3
mask = create_mask(distance_matrix, threshold)

# Create an adjacency matrix from the mask.
adjacency_matrix = create_adjacency_matrix(mask)

# Convert the adjacency matrix to a NumPy array of size 20x20.
adjacency_matrix_numpy = convert_adjacency_matrix_to_numpy(adjacency_matrix, n)

# Plot the points and the connections using the adjacency matrix.
plot_points_and_connections(points, adjacency_matrix_numpy, "Distance", "Elevation")


In [ ]:
!pip install rasterio

In [ ]:
#@title **Elevation & Distance Sample Extraction based on DENSITY and MEDIAN threshold** { vertical-output: true }
filename = "/content/processed_ATL03_20210208175352_07141002_005_01.h5" #@param {type:"string"}
track_name = "gt2r" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
start_point = 27500 #@param {type:"integer"}
distance_range = 200 #@param {type:"integer"}
elevation_band_thickness = 300 #@param {type:"integer"}


import h5py
import numpy as np
import matplotlib.pyplot as plt
import jax.numpy as jnp

def create_geometric_graph(elevations, distances, elevation_weight, distance_weight):
    # Create a complete graph with nodes representing the given points
    num_points = len(elevations)
    edges = [(i, j) for i in range(num_points) for j in range(i+1, num_points)]
    
    # Compute the weights of the edges between all pairs of nodes
    weights = np.zeros((num_points, num_points))
    for i in range(num_points):
        for j in range(i+1, num_points):
            elevation_diff = abs(elevations[i] - elevations[j])
            weights[i][j] = elevation_weight * elevation_diff + distance_weight * abs(distances[i] - distances[j])
            weights[j][i] = weights[i][j]
    
    # Create a GeometricGraph object from the edge list and weights
    geom_graph = (edges, weights)
    
    return geom_graph

def extract_elevation_data(filename, track_name, start_point, distance_range, elevation_band_thickness):
    # Open the HDF5 file and extract data
    f = h5py.File(filename, 'r')
    h_ph_data = f['/'+str(track_name)+'/heights/h_ph']
    lat_ph_data = f['/'+str(track_name)+'/heights/lat_ph']
    lon_ph_data = f['/'+str(track_name)+'/heights/lon_ph']

    # Calculate distance along track using haversine formula
    def haversine(lat1, lon1, lat2, lon2):
        R = 6371.0  # Earth radius in km
        dLat = np.radians(lat2 - lat1)
        dLon = np.radians(lon2 - lon1)
        lat1 = np.radians(lat1)
        lat2 = np.radians(lat2)
        a = np.sin(dLat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dLon/2)**2
        c = 2*np.arcsin(np.sqrt(a))
        return R*c*1000.0

    dist_ph_along_data = [haversine(lat_ph_data[0], lon_ph_data[0], lat_ph_data[i], lon_ph_data[i]) for i in range(len(lat_ph_data))]

    # Select data based on a starting point, distance range, and elevation band thickness
    dist_ph_along_data = np.array(dist_ph_along_data)
    h_ph_data = np.array(h_ph_data)

    c = np.where((dist_ph_along_data > start_point) & (dist_ph_along_data < start_point + distance_range))[0]

    median_height = np.median(h_ph_data[c])
    d = np.where((dist_ph_along_data > start_point) & (dist_ph_along_data < start_point + distance_range) & 
                 (h_ph_data > median_height - elevation_band_thickness/2) &
                 (h_ph_data < median_height + elevation_band_thickness/2))[0]

    # Extract elevation and distance data
    elevation = h_ph_data[d]
    distance = dist_ph_along_data[d]

    return distance, elevation

# filename = '/content/processed_ATL03_20200211111433_07140602_005_01.h5'
# track_name = 'gt2r'
# start_point = 25500
# distance_range = 300
# elevation_band_thickness = 50

# Create a points array.
def create_points(elevation, distance):
  return jnp.array([
    [elevation[i], distance[i]]
    for i in range(len(elevation))
  ])


def create_adjacency_matrix(points, threshold_type):
  """Creates an adjacency matrix from a set of points.

  Args:
    points: A NumPy array of points.
    threshold_type: A string that specifies the type of threshold to use.
    The possible values are:
      - "density": The threshold will be based on the density of the points.
      - "median": The threshold will be based on the median distance between all pairs of points.

  Returns:
    An adjacency matrix.
  """

  if threshold_type == "density":
    threshold = np.sqrt(len(points) / np.pi)
  elif threshold_type == "median":
    threshold = np.median(np.linalg.norm(points[:, None] - points, axis=2))
  else:
    raise ValueError("Invalid threshold type: {}".format(threshold_type))
  distance_matrix = np.linalg.norm(points[:, None] - points, axis=2)
  mask = distance_matrix < threshold
  adjacency_matrix = np.where(mask, 1, 0)
  return adjacency_matrix



# def create_adjacency_matrix(points, threshold):
#   distance_matrix = np.linalg.norm(points[:, None] - points, axis=2)
#   mask = distance_matrix < threshold
#   adjacency_matrix = np.where(mask, 1, 0)
#   return adjacency_matrix

def plot_points_and_connections(elevation, distance, threshold):
  # Create the points array.
  points = create_points(elevation, distance)
  n = len(elevation)
  adjacency_matrix = np.zeros((n, n))
  adjacency_matrix = create_adjacency_matrix(points, "density")

  # Plot the points and the connections.
  fig = plt.figure(figsize=(12, 4), dpi=200)

  for i in range(n):
    for j in range(n):
      if adjacency_matrix[i, j] == 1:
        plt.plot( [points[i, 1], points[j, 1]], [points[i, 0], points[j, 0]], 'r-')
  plt.scatter(points[:, 1], points[:, 0], s = 1)
  plt.gca().set_aspect('equal')
  plt.xlabel("X")
  plt.ylabel("Y")
  return fig

distance, elevation = extract_elevation_data(filename, track_name, start_point, distance_range, elevation_band_thickness)
threshold = "density" #@param ["density", "median"]

plot_points_and_connections(elevation, distance, threshold)
plt.show()

In [ ]:
#@title **Elevation & Distance Sample Extraction based on distance threshold in meters** { vertical-output: true }
fsize = 30
filename = "/content/processed_ATL03_20190515001514_07140302_005_01.h5" #@param {type:"string"}
track_name = "gt2l" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
start_point = 2000 #@param {type:"integer"}
distance_range = 1000 #@param {type:"integer"}
elevation_band_thickness = 500 #@param {type:"integer"}
min_height_zone = elevation_band_thickness

import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import jax.numpy as jnp

def create_geometric_graph(elevations, distances, elevation_weight, distance_weight):
  # Create a complete graph with nodes representing the given points
  num_points = len(elevations)
  edges = [(i, j) for i in range(num_points) for j in range(i+1, num_points)]
  
  # Compute the weights of the edges between all pairs of nodes
  weights = np.zeros((num_points, num_points))
  for i in range(num_points):
      for j in range(i+1, num_points):
          elevation_diff = abs(elevations[i] - elevations[j])
          weights[i][j] = elevation_weight * elevation_diff + distance_weight * abs(distances[i] - distances[j])
          weights[j][i] = weights[i][j]
  
  # Create a GeometricGraph object from the edge list and weights
  geom_graph = (edges, weights)
  
  return geom_graph

# Calculate distance along track using haversine formula
def haversine(lat1, lon1, lat2, lon2):
  R = 6371.0  # Earth radius in km
  dLat = np.radians(lat2 - lat1)
  dLon = np.radians(lon2 - lon1)
  lat1 = np.radians(lat1)
  lat2 = np.radians(lat2)
  a = np.sin(dLat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dLon/2)**2
  c = 2*np.arcsin(np.sqrt(a))
  return R*c*1000.0

w_x = 1
h_y = 1
fig, ax = plt.subplots(figsize=(30,6), dpi = 100)


def extract_elevation_data(filename, track_name, start_point, distance_range, elevation_band_thickness):
  # Open the HDF5 file and extract data
  f = h5py.File(filename, 'r')
  h_ph_data = f['/'+str(track_name)+'/heights/h_ph']
  lat_ph_data = f['/'+str(track_name)+'/heights/lat_ph']
  lon_ph_data = f['/'+str(track_name)+'/heights/lon_ph']

  dist_ph_along_data = [haversine(lat_ph_data[0], lon_ph_data[0], lat_ph_data[i], lon_ph_data[i]) for i in range(len(lat_ph_data))]

  # Select data based on a starting point, distance range, and elevation band thickness
  dist_ph_along_data = np.array(dist_ph_along_data)
  h_ph_data = np.array(h_ph_data)

      # Plot the dataset
  ax.scatter(dist_ph_along_data, h_ph_data, s=1, c = 'y')
  ax.set_xlabel('Along-Track Distance(m)', fontsize = fsize)
  ax.set_ylabel('height of photons',fontsize = fsize)
  ax.set_title('h_ph dataset from gt1l/heights',fontsize = int(fsize * 1.2 ))
  ax.set_ylim(np.median(h_ph_data) - 1500, np.median(h_ph_data) + 1500)
  
  d = get_filtered_indices(dist_ph_along_data, h_ph_data, start_point, distance_range, elevation_band_thickness)
  # Extract elevation and distance data
  elevation = h_ph_data[d]
  distance = dist_ph_along_data[d]
      
  ax.grid(True)
  ax.scatter(np.array(dist_ph_along_data)[d], np.array(h_ph_data)[d], s=2, c = 'g')
  x_mean = np.mean(np.array(dist_ph_along_data)[d])
  y_mean = np.median(np.array(h_ph_data)[d])
  ax.scatter(x_mean, y_mean, s= 40)
  ax.tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))

  rect = patches.Rectangle((x_mean-distance_range/2, y_mean-min_height_zone/2), distance_range, min_height_zone, fill=False, linestyle='dotted', linewidth=2, edgecolor='red')
  ax.add_patch(rect)
  fig.gca().set_aspect('equal')

  ax2 = fig.add_axes([0.25 - w_x/2, -0.4 - h_y/2 , w_x, h_y])
  ax2.set_xlabel('Along-Track Distance(m)', fontsize = fsize)
  ax2.set_ylabel('photon height',fontsize = fsize)
  ax2.set_ylim(y_mean-min_height_zone/2 - 10, y_mean+min_height_zone/2 + 10)
  ax2.set_title('Zoomed-in view of h_ph dataset\nfrom gt1l/heights',fontsize = int(fsize * 1.2 ))
  ax2.tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))
  ax2.grid(True)
  ax2.scatter(np.array(dist_ph_along_data)[d], np.array(h_ph_data)[d], s=20, alpha= 0.4)
  fig.gca().set_aspect('equal')

  from osgeo import gdal

  raster_location = '/content/landsat2/HLSL30.020_B04_doy2019109_aid0001_43N.tif'
  # Load the second raster image into a numpy array using gdal
  crop_ds = gdal.Open(raster_location)
  crop_band = crop_ds.GetRasterBand(1)
  crop_data = crop_band.ReadAsArray()

  # Get the mask of the second band
  crop_mask = crop_band.GetMaskBand().ReadAsArray()

  # Create a masked array from the crop data and mask
  crop_masked = np.ma.masked_where(crop_mask == 0, crop_data)

  # Get the geotransform of the second raster
  crop_gt = crop_ds.GetGeoTransform()

  # Get the minimum and maximum x and y coordinates of the second raster
  crop_xmin = crop_gt[0]
  crop_xmax = crop_gt[0] + (crop_ds.RasterXSize * crop_gt[1])
  crop_ymin = crop_gt[3] + (crop_ds.RasterYSize * crop_gt[5])
  crop_ymax = crop_gt[3]

  import rasterio
  # Open the DEM raster file
  dem_in = rasterio.open('/content/ASTER_DEM.tif')
  dem_data = dem_in.read(1, masked=True)

  extent_data = rasterio.open(raster_location)
  extent = dem_in.window(extent_data.bounds.left,  extent_data.bounds.bottom, extent_data.bounds.right, extent_data.bounds.top)

  out_data = dem_in.read(1, window=extent, masked=True)

  ax3 = fig.add_axes([0.5 - w_x / 2 , 1.4 - h_y/2 , w_x, h_y])

  ax3.set_xlabel('Longitude', fontsize = fsize)
  ax3.set_ylabel('Latitude', fontsize = fsize)
  # ax3.set_ylim(y_mean - min_height_zone/2, y_mean + min_height_zone/2)
  ax3.set_title('Zoomed-in view of h_ph dataset\nfrom gt1l/heights',fontsize = int(fsize * 1.2 ))
  ax3.tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))
  ax3.grid(True)

  ax3.imshow(out_data, extent=[crop_xmin, crop_xmax, crop_ymin, crop_ymax])

  ax3.scatter(lon_ph_data, lat_ph_data, s=1, alpha= 0.4, c ='g')
  ax3.scatter(np.array(lon_ph_data)[d], np.array(lat_ph_data)[d], s=20, alpha= 0.4, c = 'r')
  fig.gca().set_aspect('equal')

  return distance, elevation

# filename = '/content/processed_ATL03_20200211111433_07140602_005_01.h5'
# track_name = 'gt2r'
# start_point = 25500
# distance_range = 300
# elevation_band_thickness = 50

# Create a points array.
def create_points(elevation, distance):
  return jnp.array([
    [elevation[i], distance[i]]
    for i in range(len(elevation))
  ])

def get_filtered_indices(dist_ph_along_data, h_ph_data, start_point, distance_range, elevation_band_thickness):
    c = np.where((dist_ph_along_data > start_point) & (dist_ph_along_data < start_point + distance_range))[0]
    median_height = np.median(h_ph_data[c])
    d = np.where((dist_ph_along_data > start_point) & (dist_ph_along_data < start_point + distance_range) &
                 (h_ph_data > median_height - elevation_band_thickness/2) &
                 (h_ph_data < median_height + elevation_band_thickness/2))[0]
    return d

def create_adjacency_matrix(points, threshold):
  distance_matrix = np.linalg.norm(points[:, None] - points, axis=2)
  mask = distance_matrix < threshold
  adjacency_matrix = np.where(mask, 1, 0)
  return adjacency_matrix

def plot_points_and_connections(elevation, distance, threshold):
  # Create the points array.
  points = create_points(elevation, distance)
  n = len(elevation)
  adjacency_matrix = np.zeros((n, n))
  adjacency_matrix = create_adjacency_matrix(points, threshold)

  # Plot the points and the connections.
  ax4 = fig.add_axes([0.75 -w_x/2 , -0.4 - h_y/2 , w_x, h_y])
  # fig = plt.figure(figsize=(12, 4), dpi=200)
  ax4.set_title('Nearest Neighbour connection view',fontsize = int(fsize * 1.2 ))

  for i in range(n):
    for j in range(n):
      if adjacency_matrix[i, j] == 1:
        ax4.plot( [points[i, 1], points[j, 1]], [points[i, 0], points[j, 0]], 'r-')
  ax4.scatter(points[:, 1], points[:, 0], s = 1)
  fig.gca().set_aspect('equal')
  ax4.tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))
  ax4.grid(True)
  ax4.set_xlabel('distance', fontsize = fsize)
  ax4.set_ylabel('elevation', fontsize = fsize)
  # return fig

distance, elevation = extract_elevation_data(filename, track_name, start_point, distance_range, elevation_band_thickness)
threshold = 3 #@param {type:"integer"}
plot_points_and_connections(elevation, distance, threshold)

raster_file1 = '/content/landsat2/HLSL30.020_B04_doy2019109_aid0001_43N.tif'
raster_file2 = '/content/landsat2/HLSL30.020_B05_doy2019109_aid0001_43N.tif'
raster_file3 = '/content/landsat2/HLSL30.020_B06_doy2019109_aid0001_43N.tif'

# Open the HDF5 file and extract data
f = h5py.File(filename, 'r')
h_ph_data = f['/'+str(track_name)+'/heights/h_ph']
lat_ph_data = f['/'+str(track_name)+'/heights/lat_ph']
lon_ph_data = f['/'+str(track_name)+'/heights/lon_ph']

dist_ph_along_data = [haversine(lat_ph_data[0], lon_ph_data[0], lat_ph_data[i], lon_ph_data[i]) for i in range(len(lat_ph_data))]
d = get_filtered_indices(np.array(dist_ph_along_data), np.array(h_ph_data), start_point, distance_range, elevation_band_thickness)
latitudes = lat_ph_data[d]
longitudes = lon_ph_data[d]
distance = np.array(dist_ph_along_data)[d]
print(len(distance))

def plot_reflectance_by_band(raster_file1, raster_file2, raster_file3, latitudes, longitudes, distance):
  # Load raster file
  ax5 = fig.add_axes([0.5 -w_x/2 , -1.4 - h_y/2 , w_x, h_y])

  print(len(distance), "<<here>>", len(latitudes))
  src1 = rasterio.open(raster_file1)
  src2 = rasterio.open(raster_file2)
  src3 = rasterio.open(raster_file3)

  crs = src1.crs
  transform = src1.transform

  values1 = [x[0] for x in src1.sample(zip(latitudes, longitudes))]
  values2 = [x[0] for x in src2.sample(zip(latitudes, longitudes))]
  values3 = [x[0] for x in src3.sample(zip(latitudes, longitudes))]
  # Normalize the values to be in the range [0, 1]
  values1 = np.interp(values1, (np.min(values1), np.max(values1)), (0, 100))
  values2 = np.interp(values2, (np.min(values2), np.max(values2)), (0, 100))
  values3 = np.interp(values3, (np.min(values3), np.max(values3)), (0, 100))


  ax5.scatter(distance, values1, c='r', s=1, alpha = 0.3)
  ax5.scatter(distance, values2, c='g', s=1, alpha = 0.3)
  ax5.scatter(distance, values3, c='b', s=1, alpha = 0.3)
  fig.gca().set_aspect('equal')
  ax5.tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))
  ax5.grid(True)
  ax5.set_xlabel('distance', fontsize = fsize)
  ax5.set_ylabel('elevation', fontsize = fsize)
  ax5.set_ylim(0, 100)

# plot_reflectance_by_band(raster_file1, raster_file2, raster_file3, latitudes, longitudes, distance)
# plt.show()

In [ ]:
fsize = 30
filename = "/content/processed_ATL03_20200531180255_10110706_005_01.h5" #@param {type:"string"}
track_name = "gt2l" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
start_point = 35000 #@param {type:"integer"}
distance_range = 1000 #@param {type:"integer"}
elevation_band_thickness = 500 #@param {type:"integer"}
min_height_zone = elevation_band_thickness
def extract_elevation_data(filename, track_name, start_point, distance_range, elevation_band_thickness):
  # Open the HDF5 file and extract data
  f = h5py.File(filename, 'r')
  h_ph_data = f['/'+str(track_name)+'/heights/h_ph']
  lat_ph_data = f['/'+str(track_name)+'/heights/lat_ph']
  lon_ph_data = f['/'+str(track_name)+'/heights/lon_ph']

  dist_ph_along_data = [haversine(lat_ph_data[0], lon_ph_data[0], lat_ph_data[i], lon_ph_data[i]) for i in range(len(lat_ph_data))]

  # Select data based on a starting point, distance range, and elevation band thickness
  dist_ph_along_data = np.array(dist_ph_along_data)
  h_ph_data = np.array(h_ph_data)

In [ ]:
filename = "/content/processed_ATL03_20200531180255_10110706_005_01.h5" #@param {type:"string"}
track_name = "gt2l" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
def get_height_lat_lon(filename, track_name):
  # Open the HDF5 file and extract data
  f = h5py.File(filename, 'r')
  h_ph_data = f['/'+str(track_name)+'/heights/h_ph']
  lat_ph_data = f['/'+str(track_name)+'/heights/lat_ph']
  lon_ph_data = f['/'+str(track_name)+'/heights/lon_ph']
  return h_ph_data, lat_ph_data, lon_ph_data

In [ ]:
#@title  { vertical-output: true }
filename_1 = "/content/processed_ATL03_20190213043533_07140202_005_01.h5" #@param {type:"string"}
tr_1 = "gt1l" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
tr_2 = "gt1r" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
tr_3 = "gt2l" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
tr_4 = "gt2r" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
tr_5 = "gt3l" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
tr_6 = "gt3r" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
filename_2 = "/content/processed_ATL03_20200531180255_10110706_005_01.h5" #@param {type:"string"}
track_name_2 = "gt1l" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]


from osgeo import gdal

raster_location = '/content/pongdam_dem.tif'
# Load the second raster image into a numpy array using gdal
crop_ds = gdal.Open(raster_location)
crop_band = crop_ds.GetRasterBand(1)
crop_data = crop_band.ReadAsArray()

# Get the mask of the second band
crop_mask = crop_band.GetMaskBand().ReadAsArray()

# Create a masked array from the crop data and mask
crop_masked = np.ma.masked_where(crop_mask == 0, crop_data)

# Get the geotransform of the second raster
crop_gt = crop_ds.GetGeoTransform()

# Get the minimum and maximum x and y coordinates of the second raster
crop_xmin = crop_gt[0]
crop_xmax = crop_gt[0] + (crop_ds.RasterXSize * crop_gt[1])
crop_ymin = crop_gt[3] + (crop_ds.RasterYSize * crop_gt[5])
crop_ymax = crop_gt[3]

import rasterio
# Open the DEM raster file
dem_in = rasterio.open('/content/pongdam_dem.tif')
dem_data = dem_in.read(1, masked=True)

extent_data = rasterio.open(raster_location)
extent = dem_in.window(extent_data.bounds.left,  extent_data.bounds.bottom, extent_data.bounds.right, extent_data.bounds.top)

out_data = dem_in.read(1, window=extent, masked=1)

w_x = 1
h_y = 1
fig, ax = plt.subplots(figsize=(30,6), dpi = 100)
fig.gca().set_aspect('equal')

ax.set_xlabel('Longitude', fontsize = fsize)
ax.set_ylabel('Latitude', fontsize = fsize)
# ax3.set_ylim(y_mean - min_height_zone/2, y_mean + min_height_zone/2)
ax.set_title('DEM view',fontsize = int(fsize * 1.2 ))
ax.tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))
ax.grid(True)

ax.imshow(out_data, extent=[crop_xmin, crop_xmax, crop_ymin, crop_ymax])
# lon, lat = get_height_lat_lon(filename_1, tr_1)[2], get_height_lat_lon(filename_1, tr_1)[1]
# ax.scatter(lon, lat, s=1, alpha= 1, c ='aqua', label= tr_1)
# lon, lat = get_height_lat_lon(filename_1, tr_2)[2], get_height_lat_lon(filename_1, tr_2)[1]
# ax.scatter(lon, lat, s=1, alpha= 1, c ='violet', label= tr_2)
# lon, lat = get_height_lat_lon(filename_1, tr_3)[2], get_height_lat_lon(filename_1, tr_3)[1]
# ax.scatter(lon, lat, s=1, alpha= 1, c ='deeppink', label= tr_3)
# lon, lat = get_height_lat_lon(filename_1, tr_4)[2], get_height_lat_lon(filename_1, tr_4)[1]
# ax.scatter(lon, lat, s=1, alpha= 1, c ='gold', label= tr_4)
# lon, lat = get_height_lat_lon(filename_1, tr_5)[2], get_height_lat_lon(filename_1, tr_5)[1]
# ax.scatter(lon, lat, s=1, alpha= 1, c ='lime', label= tr_5)
lon, lat = get_height_lat_lon(filename_1, tr_6)[2], get_height_lat_lon(filename_1, tr_6)[1]
ax.scatter(lon, lat, s=1, alpha= 1, c ='springgreen', label= tr_6)

lon, lat = get_height_lat_lon(filename_2, tr_1)[2], get_height_lat_lon(filename_2, tr_1)[1]
ax.scatter(lon, lat, s=1, alpha= 1, c ='royalblue', label= tr_1)
# lon, lat = get_height_lat_lon(filename_2, tr_2)[2], get_height_lat_lon(filename_2, tr_2)[1]
# ax.scatter(lon, lat, s=1, alpha= 1, c ='lightyellow', label= tr_2)
# lon, lat = get_height_lat_lon(filename_2, tr_3)[2], get_height_lat_lon(filename_2, tr_3)[1]
# ax.scatter(lon, lat, s=1, alpha= 1, c ='red', label= tr_3)
# lon, lat = get_height_lat_lon(filename_2, tr_4)[2], get_height_lat_lon(filename_2, tr_4)[1]
# ax.scatter(lon, lat, s=1, alpha= 1, c ='greenyellow', label= tr_4)
# lon, lat = get_height_lat_lon(filename_2, tr_5)[2], get_height_lat_lon(filename_2, tr_5)[1]
# ax.scatter(lon, lat, s=1, alpha= 1, c ='lawngreen', label= tr_5)
# lon, lat = get_height_lat_lon(filename_2, tr_6)[2], get_height_lat_lon(filename_2, tr_6)[1]
# ax.scatter(lon, lat, s=1, alpha= 0.4, c ='lightyellow', label= tr_6)






h_ph_data1, lat1, lon1 = get_height_lat_lon(filename_1, tr_6)
h_ph_data2, lat2, lon2 = get_height_lat_lon(filename_2, tr_1)

def get_filtered_indices(dist_ph_along_data, h_ph_data, start_point, distance_range, elevation_band_thickness):
    c = np.where((dist_ph_along_data > start_point) & 
                 (dist_ph_along_data < start_point + distance_range))[0]
    median_height = np.median(h_ph_data[c])
    d = np.where((dist_ph_along_data > start_point) & 
                 (dist_ph_along_data < start_point + distance_range) &
                 (h_ph_data > median_height - elevation_band_thickness/2) &
                 (h_ph_data < median_height + elevation_band_thickness/2))[0]
    return d
# lon_ph_data, lat_ph_data = 
dist_ph_along_data1 = [haversine(lat1[0], lon1[0], lat1[i], lon1[i]) for i in range(len(lat1))]
dist_ph_along_data1 = np.array(dist_ph_along_data1)
h_ph_data1 = np.array(h_ph_data1)
start_point = 12000 #@param {type:"integer"}
distance_range = 1000 #@param {type:"integer"}
elevation_band_thickness = 500 #@param {type:"integer"}
d = get_filtered_indices(dist_ph_along_data1, h_ph_data1, start_point, distance_range, elevation_band_thickness)
ax.scatter(lon1[d], lat1[d], s=1, alpha= 1, c ='r', label= "track 1")

dist_ph_along_data2 = [haversine(lat2[0], lon2[0], lat2[i], lon2[i]) for i in range(len(lat2))]
dist_ph_along_data2 = np.array(dist_ph_along_data2)
h_ph_data2 = np.array(h_ph_data2)
start_point = 20000 #@param {type:"integer"}
distance_range = 1000 #@param {type:"integer"}
elevation_band_thickness = 500 #@param {type:"integer"}
d = get_filtered_indices(dist_ph_along_data2, h_ph_data2, start_point, distance_range, elevation_band_thickness)
ax.scatter(lon2[d], lat2[d], s=1, alpha= 1, c ='violet', label= "track 2")

ax.legend()


In [ ]:
def get_filtered_LatLon(lat, lon, lat_ranging, lon_ranging):
  d = np.where((lon < max(lon_ranging)) & 
                (lon > min(lon_ranging)) &
                (lat < max(lon_ranging)) &
                (lat > min(lon_ranging))
                )[0]
  return d

In [ ]:
#@title  { vertical-output: true }
w_x = 1
h_y = 1
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize=(30,6), dpi = 100)  
ax[0].set_xlabel('Along-Track Distance(m)', fontsize = fsize)
ax[0].set_ylabel('photon height',fontsize = fsize)

dist_ph_along_data1 = [haversine(lat1[0], lon1[0], lat1[i], lon1[i]) for i in range(len(lat1))]
dist_ph_along_data1 = np.array(dist_ph_along_data1)
h_ph_data1 = np.array(h_ph_data1)
start_point = 20000 #@param {type:"integer"}
distance_range = 1000 #@param {type:"integer"}
elevation_band_thickness = 500 #@param {type:"integer"}
d = get_filtered_indices(dist_ph_along_data1, h_ph_data1, start_point, distance_range, elevation_band_thickness)
d_1 = d


x_mean = np.mean(np.array(dist_ph_along_data1)[d])
y_mean = np.median(np.array(h_ph_data1)[d])
ax[0].set_ylim(y_mean-min_height_zone/2 - 10, y_mean+min_height_zone/2 + 10)
ax[0].set_title('Zoomed-in view of h_ph dataset\nfrom gt1l/heights',fontsize = int(fsize * 1.2 ))
ax[0].tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))
ax[0].grid(True)
d1 = np.array(dist_ph_along_data1)[d]
h1 = np.array(h_ph_data1)[d]
ax[0].scatter(np.array(dist_ph_along_data1)[d], np.array(h_ph_data1)[d], s=20, alpha= 0.4)
fig.gca().set_aspect('equal')
##########################################################################################################
w_x = 1
h_y = 1

ax[1].set_xlabel('Along-Track Distance(m)', fontsize = fsize)
ax[1].set_ylabel('photon height',fontsize = fsize)

dist_ph_along_data2 = [haversine(lat2[0], lon2[0], lat2[i], lon2[i]) for i in range(len(lat2))]
dist_ph_along_data2 = np.array(dist_ph_along_data2)
h_ph_data2 = np.array(h_ph_data2)
start_point = 12000 #@param {type:"integer"}
distance_range = 1000 #@param {type:"integer"}
elevation_band_thickness = 500 #@param {type:"integer"}
d = get_filtered_indices(dist_ph_along_data2, h_ph_data2, start_point, distance_range, elevation_band_thickness)



x_mean = np.mean(np.array(dist_ph_along_data2)[d])
y_mean = np.median(np.array(h_ph_data2)[d])
ax[1].set_ylim(y_mean-min_height_zone/2 - 10, y_mean+min_height_zone/2 + 10)
ax[1].set_title('Zoomed-in view of h_ph dataset\nfrom gt1l/heights',fontsize = int(fsize * 1.2 ))
ax[1].tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))
ax[1].grid(True)

ax[1].scatter(np.array(dist_ph_along_data2)[d], np.array(h_ph_data2)[d], s=20, alpha= 0.4)
fig.gca().set_aspect('equal')

fig1, ax = plt.subplots(figsize=(30,6), dpi = 100)  
ax.set_xlabel('Along-Track Distance(m)', fontsize = fsize)
ax.set_ylabel('photon height',fontsize = fsize)
ax.grid(True)
dx = get_filtered_LatLon(np.array(lat2), np.array(lon2), np.array(lat1)[d_1], np.array(lon1)[d_1])
d2 = np.array(dist_ph_along_data1)[dx]
h2 = np.array(h_ph_data2)[dx]
ax.scatter(d1, h1, s=1, c = 'r', alpha = 1)
ax.scatter(d2, h2, s=1, c = 'g', alpha = 1)
fig1.gca().set_aspect('equal')


test

In [ ]:
#@title **tracks 1 & 2 elevation vs distance plots**{ vertical-output: true }
import numpy as np
import matplotlib.pyplot as plt

# Define parameters
start_point_1, distance_range_1, elevation_band_thickness_1 = 20000, 1000, 500
start_point_2, distance_range_2, elevation_band_thickness_2 = 12000, 1000, 500

# Calculate distances
dist_ph_along_data1 = np.array([haversine(lat1[0], lon1[0], lat1[i], lon1[i]) for i in range(len(lat1))])
dist_ph_along_data2 = np.array([haversine(lat2[0], lon2[0], lat2[i], lon2[i]) for i in range(len(lat2))])

# Filter data
d1 = get_filtered_indices(dist_ph_along_data1, h_ph_data1, start_point_1, distance_range_1, elevation_band_thickness_1)
d2 = get_filtered_indices(dist_ph_along_data2, h_ph_data2, start_point_2, distance_range_2, elevation_band_thickness_2)
dx = get_filtered_LatLon(np.array(lat2), np.array(lon2), np.array(lat1)[d1], np.array(lon1)[d1])

# Create plots
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(30, 6), dpi=100)
axs[0].set_xlabel('Along-Track Distance(m)', fontsize=fsize)
axs[0].set_ylabel('photon height', fontsize=fsize)
axs[0].set_ylim(np.median(h_ph_data1[d1]) - min_height_zone/2 - 10, np.median(h_ph_data1[d1]) + min_height_zone/2 + 10)
axs[0].set_title('Zoomed-in view of h_ph dataset\nfrom gt1l/heights', fontsize=int(fsize * 1.2))
axs[0].tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))
axs[0].grid(True)
axs[0].scatter(dist_ph_along_data1[d1], h_ph_data1[d1], s=20, alpha=0.4)
axs[0].set_aspect('equal')

axs[1].set_xlabel('Along-Track Distance(m)', fontsize=fsize)
axs[1].set_ylabel('photon height', fontsize=fsize)
axs[1].set_ylim(np.median(h_ph_data2[d2]) - min_height_zone/2 - 10, np.median(h_ph_data2[d2]) + min_height_zone/2 + 10)
axs[1].set_title('Zoomed-in view of h_ph dataset\nfrom gt1l/heights', fontsize=int(fsize * 1.2))
axs[1].tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))
axs[1].grid(True)
axs[1].scatter(dist_ph_along_data2[d2], h_ph_data2[d2], s=20, alpha=0.4)
axs[1].set_aspect('equal')

fig1, ax = plt.subplots(figsize=(30, 6), dpi=100)
ax.set_xlabel('Along-Track Distance(m)', fontsize=fsize)
ax.set_ylabel('photon height', fontsize=fsize)
ax.grid(True)
ax.scatter(dist_ph_along_data1[d1], h_ph_data1[d1], s=1, c='r', alpha=1)
ax.scatter(np.array(dist_ph_along_data2)[d], np.array(h_ph_data2)[d], s=20, alpha=0.4)
fig.gca().set_aspect('equal')

fig1, ax = plt.subplots(figsize=(30, 6), dpi=100)
ax.set_xlabel('Along-Track Distance(m)', fontsize=fsize)
ax.set_ylabel('photon height', fontsize=fsize)
ax.grid(True)
dx = get_filtered_LatLon(np.array(lat2), np.array(lon2), np.array(lat1)[d_1], np.array(lon1)[d_1])
d2 = np.array(dist_ph_along_data1)[dx]
h2 = np.array(h_ph_data2)[dx]
ax.scatter(d1, h1, s=1, c='r', alpha=1)
ax.scatter(d2, h2, s=1, c='g', alpha=1)
fig1.gca().set_aspect('equal')


In [ ]:
#@title **track intersection plot**{ vertical-output: true }
lat_1 = np.array(lat1)
lon_1 = np.array(lon1)
lat_2 = np.array(lat2)
lon_2 = np.array(lon2)

# Find the bounding box that covers both tracks with a buffer
min_lon = min(min(lon_1), min(lon_2)) - 0.05
max_lon = max(max(lon_1), max(lon_2)) + 0.05
min_lat = min(min(lat_1), min(lat_2)) - 0.05
max_lat = max(max(lat_1), max(lat_2)) + 0.05

# Create a new figure and axis
fig, ax = plt.subplots(figsize=(8, 8))

# Plot the tracks separately on the new axis
ax.plot(lon_1, lat_1, '.', color='blue', markersize=1)
ax.plot(lon_2, lat_2, '.', color='red', markersize=1)

# Set the limits of the axis to the bounding box
ax.set_xlim(min_lon, max_lon)
ax.set_ylim(min_lat, max_lat)

# Add a grid to the plot
ax.grid(True)

# Add a legend to the plot
ax.legend(['Track 1', 'Track 2'], loc='lower right')

# Add axis labels and a title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Tracks 1 and 2')


In [ ]:
print(len(lat1),len(lat_1))

In [ ]:
#@title **track elevation vs lat/long chart plot**{ vertical-output: true }
# Create a new figure with two subplots
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,6))

# Plot the elevation vs latitude for both tracks on the first subplot
ax1.plot(lat_1, h_ph_data1, '.', color='blue', markersize=1)
ax1.plot(lat_2, h_ph_data2, '.', color='red', markersize=1)
ax1.set_xlabel('Latitude')
ax1.set_ylabel('Elevation (m)')
ax1.set_title('Elevation vs Latitude')
ax1.legend(['Track 1', 'Track 2'], loc='upper right')
ax1.tick_params(axis='both', which='major', labelsize=8)
ax1.grid(True)

# Plot the elevation vs longitude for both tracks on the second subplot
ax2.plot(lon_1, h_ph_data1, '.', color='blue', markersize=1)
ax2.plot(lon_2, h_ph_data2, '.', color='red', markersize=1)
ax2.set_xlabel('Longitude')
ax2.set_ylabel('Elevation (m)')
ax2.set_title('Elevation vs Longitude')
ax2.legend(['Track 1', 'Track 2'], loc='upper right')
ax2.tick_params(axis='both', which='major', labelsize=8)
ax2.grid(True)

plt.tight_layout()
plt.show()



In [ ]:
#@title **tracks buffer plot with its intersections at centre**{ vertical-output: true }
from shapely.geometry import Point
from shapely.ops import unary_union
from shapely.geometry import LineString

# Convert the latitudes and longitudes to shapely Point objects
points_1 = [Point(lon, lat) for lat, lon in zip(lat_1, lon_1)]
points_2 = [Point(lon, lat) for lat, lon in zip(lat_2, lon_2)]

# Find the intersection of the two tracks
intersection = unary_union([LineString(points_1).intersection(LineString(points_2))])
buffer_region = 0.002 #@param {type:"raw"}

# Create a buffer around the intersection point
intersection_buffer = intersection.buffer(buffer_region)

# Create a new figure and axis
fig, ax = plt.subplots(figsize=(8, 8))

# Plot the tracks separately on the new axis
ax.plot(lon_1, lat_1, '.', color='blue', markersize=1)
ax.plot(lon_2, lat_2, '.', color='red', markersize=1)

# Plot the intersection buffer
x, y = intersection_buffer.exterior.xy
ax.fill(x, y, alpha=0.3, fc='green', ec='none')

# Set the limits of the axis to the bounding box
ax.set_xlim(min_lon, max_lon)
ax.set_ylim(min_lat, max_lat)

# Add a grid to the plot
ax.grid(True)

# Add a legend to the plot
ax.legend(['Track 1', 'Track 2'], loc='lower right')

# Add axis labels and a title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Tracks 1 and 2 with Intersection Buffer')


In [ ]:
print(len(points_1))

In [ ]:
#@title **tracks altitude within the buffered region**{ vertical-output: true }
import geopandas as gpd

# Create a GeoDataFrame for the buffered intersection
intersection_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(intersection_buffer))

# Create a mask for selecting points within the buffered region
mask_1 = [intersection_buffer.contains(point) for point in points_1]
mask_2 = [intersection_buffer.contains(point) for point in points_2]

# Select the latitudes, longitudes, and heights within the buffered region for each track
lat_1_buf = lat_1[mask_1]
lon_1_buf = lon_1[mask_1]
height_1_buf = h_ph_data1[mask_1]

lat_2_buf = lat_2[mask_2]
lon_2_buf = lon_2[mask_2]
height_2_buf = h_ph_data2[mask_2]

# Create a new figure and axis for the height vs. lat and height vs. lon plots side by side
fig, axs = plt.subplots(ncols=2, figsize=(16, 6))

# Plot the height vs. lat values for each track within the buffered region on the first axis
axs[0].plot(lat_1_buf, height_1_buf, '.', color='blue', markersize=1)
axs[0].plot(lat_2_buf, height_2_buf, '.', color='red', markersize=1)

# Add axis labels and a title to the first axis
axs[0].set_xlabel('Latitude')
axs[0].set_ylabel('Height')
axs[0].set_title('Height vs. Latitude for Tracks 1 and 2 within Buffered Region')

# Plot the height vs. lon values for each track within the buffered region on the second axis
axs[1].plot(lon_1_buf, height_1_buf, '.', color='blue', markersize=1)
axs[1].plot(lon_2_buf, height_2_buf, '.', color='red', markersize=1)

# Add axis labels and a title to the second axis
axs[1].set_xlabel('Longitude')
axs[1].set_ylabel('Height')
axs[1].set_title('Height vs. Longitude for Tracks 1 and 2 within Buffered Region')

# Add a title to the figure
fig.suptitle('Height vs. Latitude and Longitude for Tracks 1 and 2 within Buffered Region')

# Show the plot
plt.show()


In [ ]:
#@title  **track altitudes within the buffered region with buffered altitude/elevation axis**{ vertical-output: true }
# Compute the median height of the tracks within the buffered region
median_height = np.median(np.concatenate((height_1_buf, height_2_buf)))

# Define the upper and lower bounds of the elevation buffer
upper_bound = median_height + 50
lower_bound = median_height - 50

# Create a new figure and axis for the height vs. lat plot
fig, ax = plt.subplots(figsize=(8, 6))

# Plot the height vs. lat values for each track within the buffered region
ax.plot(lat_1_buf, height_1_buf, '.', color='blue', markersize=1)
ax.plot(lat_2_buf, height_2_buf, '.', color='red', markersize=1)

# Add the elevation buffer to the plot
ax.axhspan(lower_bound, upper_bound, color='green', alpha=0.3)

# Add axis labels and a title
ax.set_xlabel('Latitude')
ax.set_ylabel('Height')
ax.set_title('Height vs. Latitude for Tracks 1 and 2 within Buffered Region')

# Create a new figure and axis for the height vs. lon plot
fig, ax = plt.subplots(figsize=(8, 6))

# Plot the height vs. lon values for each track within the buffered region
ax.plot(lon_1_buf, height_1_buf, '.', color='blue', markersize=1)
ax.plot(lon_2_buf, height_2_buf, '.', color='red', markersize=1)

# Add the elevation buffer to the plot
ax.axhspan(lower_bound, upper_bound, color='green', alpha=0.3)

# Add axis labels and a title
ax.set_xlabel('Longitude')
ax.set_ylabel('Height')
ax.set_title('Height vs. Longitude for Tracks 1 and 2 within Buffered Region')
# Calculate the buffer distance for the y-axis limits
# Calculate the median elevation
median_elevation = np.median(np.concatenate((height_1_buf, height_2_buf)))

# Calculate the buffer distance for the y-axis limits
y_buffer = 50

# Set the y-axis limits to the buffered distance above and below the median elevation
ax.set_ylim([median_elevation - y_buffer, median_elevation + y_buffer])


In [ ]:
#@title  **track in the buffered region with altitude/elevatoion/height buffer control for points visibility**{ vertical-output: true }
# Compute the median height of the tracks within the buffered region
median_height = np.median(np.concatenate((height_1_buf, height_2_buf)))
buffer_height = 5 #@param {type:"raw"}

# Define the upper and lower bounds of the elevation buffer
upper_bound = median_height + buffer_height
lower_bound = median_height - buffer_height

# Create a new figure and axis for the height vs. lat plot
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot the height vs. lat values for each track within the buffered region for the first subplot
axs[0].plot(lat_1_buf, height_1_buf, '.', color='blue', markersize=1)
axs[0].plot(lat_2_buf, height_2_buf, '.', color='red', markersize=1)

# Add the elevation buffer to the plot
axs[0].axhspan(lower_bound, upper_bound, color='green', alpha=0.3)

# Set the y-axis limits to the buffered distance above and below the median elevation
axs[0].set_ylim([median_height - buffer_height, median_height + buffer_height])

# Add axis labels and a title to the first subplot
axs[0].set_xlabel('Latitude')
axs[0].set_ylabel('Height')
axs[0].set_title('Height vs. Latitude for Tracks 1 and 2 within Buffered Region')

# Plot the height vs. lon values for each track within the buffered region for the second subplot
axs[1].plot(lon_1_buf, height_1_buf, '.', color='blue', markersize=1)
axs[1].plot(lon_2_buf, height_2_buf, '.', color='red', markersize=1)

# Add the elevation buffer to the plot
axs[1].axhspan(lower_bound, upper_bound, color='green', alpha=0.3)

# Set the y-axis limits to the buffered distance above and below the median elevation
axs[1].set_ylim([median_height - buffer_height, median_height + buffer_height])

# Add axis labels and a title to the second subplot
axs[1].set_xlabel('Longitude')
axs[1].set_ylabel('Height')
axs[1].set_title('Height vs. Longitude for Tracks 1 and 2 within Buffered Region')

# Show the plot
plt.show()


In [ ]:
#@title **track in the buffered region with altitude/elevatoion/height buffer control for points visibility with track labels** { vertical-output: true }
# Compute the median height of the tracks within the buffered region
median_height = np.median(np.concatenate((height_1_buf, height_2_buf)))
buffer_height = 5 #@param {type:"raw"}

# Define the upper and lower bounds of the elevation buffer
upper_bound = median_height + buffer_height
lower_bound = median_height - buffer_height

# Create a new figure and axis for the height vs. lat plot
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot the height vs. lat values for each track within the buffered region for the first subplot
axs[0].plot(lat_1_buf, height_1_buf, '.', color='blue', markersize=1, label='Track 1')
axs[0].plot(lat_2_buf, height_2_buf, '.', color='red', markersize=1, label='Track 2')

# Add the elevation buffer to the plot
axs[0].axhspan(lower_bound, upper_bound, color='green', alpha=0.3)

# Set the y-axis limits to the buffered distance above and below the median elevation
axs[0].set_ylim([median_height - buffer_height, median_height + buffer_height])

# Add axis labels and a title to the first subplot
axs[0].set_xlabel('Latitude')
axs[0].set_ylabel('Height')
axs[0].set_title('Height vs. Latitude for Tracks 1 and 2 within Buffered Region')

# Add a legend to the first subplot
axs[0].legend()

# Plot the height vs. lon values for each track within the buffered region for the second subplot
axs[1].plot(lon_1_buf, height_1_buf, '.', color='blue', markersize=1, label='Track 1')
axs[1].plot(lon_2_buf, height_2_buf, '.', color='red', markersize=1, label='Track 2')

# Add the elevation buffer to the plot
axs[1].axhspan(lower_bound, upper_bound, color='green', alpha=0.3)

# Set the y-axis limits to the buffered distance above and below the median elevation
axs[1].set_ylim([median_height - buffer_height, median_height + buffer_height])

# Add axis labels and a title to the second subplot
axs[1].set_xlabel('Longitude')
axs[1].set_ylabel('Height')
axs[1].set_title('Height vs. Longitude for Tracks 1 and 2 within Buffered Region')

# Add a legend to the second subplot
axs[1].legend()

# Show the plot
plt.show()


In [ ]:
#@title **tracks buffer plot with its intersections at centre**{ vertical-output: true }
from shapely.geometry import Point
from shapely.ops import unary_union
from shapely.geometry import LineString

# Convert the latitudes and longitudes to shapely Point objects
points_1 = [Point(lon, lat) for lat, lon in zip(lat_1, lon_1)]
points_2 = [Point(lon, lat) for lat, lon in zip(lat_2, lon_2)]

# Find the intersection of the two tracks
intersection = unary_union([LineString(points_1).intersection(LineString(points_2))])
buffer_region = 0.002 #@param {type:"raw"}

# Create a buffer around the intersection point
intersection_buffer = intersection.buffer(buffer_region)
#@title **track in the buffered region with altitude/elevatoion/height buffer control for points visibility with track labels** { vertical-output: true }
# Compute the median height of the tracks within the buffered region
import geopandas as gpd

# Create a GeoDataFrame for the buffered intersection
intersection_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(intersection_buffer))

# Create a mask for selecting points within the buffered region
mask_1 = [intersection_buffer.contains(point) for point in points_1]
mask_2 = [intersection_buffer.contains(point) for point in points_2]

# Select the latitudes, longitudes, and heights within the buffered region for each track
lat_1_buf = lat_1[mask_1]
lon_1_buf = lon_1[mask_1]
height_1_buf = h_ph_data1[mask_1]

lat_2_buf = lat_2[mask_2]
lon_2_buf = lon_2[mask_2]
height_2_buf = h_ph_data2[mask_2]

##########################################################
# Set the limits of the axis to the bounding box
# Find the bounding box that covers both tracks with a buffer
min_lon = min(min(lon_1_buf), min(lon_2_buf)) - 0.001
max_lon = max(max(lon_1_buf), max(lon_2_buf)) + 0.001
min_lat = min(min(lat_1_buf), min(lat_2_buf)) - 0.001
max_lat = max(max(lat_1_buf), max(lat_2_buf)) + 0.001
M_lon = (max_lon + min_lon)/2
M_lat = (max_lat + min_lat)/2
dx_boundary = max(max_lon - min_lon, max_lat - min_lat)
###########################################################
# Create a new figure and axis
fig, ax = plt.subplots(figsize=(8, 8))

# Plot the tracks separately on the new axis
ax.plot(lon_1, lat_1, '.', color='blue', markersize=1)
ax.plot(lon_2, lat_2, '.', color='red', markersize=1)

# Plot the intersection buffer
x, y = intersection_buffer.exterior.xy
ax.fill(x, y, alpha=1, fc='#b6fa64', ec='none')

ax.set_xlim(M_lon - dx_boundary/2, M_lon + dx_boundary/2)
ax.set_ylim(M_lat - dx_boundary/2, M_lat + dx_boundary/2)
# Add a grid to the plot
ax.grid(True)

# Add a legend to the plot
ax.legend(['Track 1', 'Track 2'], loc='lower right')

# Add axis labels and a title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Tracks 1 and 2 with Intersection Buffer')




median_height = np.median(np.concatenate((height_1_buf, height_2_buf)))
buffer_height = 3 #@param {type:"raw"}

# Define the upper and lower bounds of the elevation buffer
upper_bound = median_height + buffer_height
lower_bound = median_height - buffer_height

# Create a new figure and axis for the height vs. lat plot
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# Plot the height vs. lat values for each track within the buffered region for the first subplot
axs[0].plot(lat_1_buf, height_1_buf, '.', color='blue', markersize=1, label='Track 1')
axs[0].plot(lat_2_buf, height_2_buf, '.', color='red', markersize=1, label='Track 2')

# Add the elevation buffer to the plot
axs[0].axhspan(lower_bound, upper_bound, color='#fdffa3', alpha=0.3)

# Set the y-axis limits to the buffered distance above and below the median elevation
axs[0].set_ylim([median_height - buffer_height, median_height + buffer_height])

# Add axis labels and a title to the first subplot
axs[0].set_xlabel('Latitude')
axs[0].set_ylabel('Height')
axs[0].set_title('Height vs. Latitude for Tracks 1 and 2 within Buffered Region')

# Add a legend to the first subplot
axs[0].legend()

# Plot the height vs. lon values for each track within the buffered region for the second subplot
axs[1].plot(lon_1_buf, height_1_buf, '.', color='blue', markersize=1, label='Track 1')
axs[1].plot(lon_2_buf, height_2_buf, '.', color='red', markersize=1, label='Track 2')

# Add the elevation buffer to the plot
axs[1].axhspan(lower_bound, upper_bound, color='#fdffa3', alpha=0.3)

# Set the y-axis limits to the buffered distance above and below the median elevation
axs[1].set_ylim([median_height - buffer_height, median_height + buffer_height])

# Add axis labels and a title to the second subplot
axs[1].set_xlabel('Longitude')
axs[1].set_ylabel('Height')
axs[1].set_title('Height vs. Longitude for Tracks 1 and 2 within Buffered Region')

# Add a legend to the second subplot
axs[1].legend()

# Show the plot
plt.show()


In [ ]:
print(len(lat_1_buf), len(lon_1_buf))
fig, ax = plt.subplots(figsize=(16, 6))

dist_ph_along_data_1 = [haversine(lat_1_buf[0], lon_1_buf[0], lat_1_buf[i], lon_1_buf[i]) for i in range(len(lat_1_buf))]

lats = np.flip(lat_2_buf)
longs = np.flip(lon_2_buf)
heights = np.flip(height_2_buf)

dist_ph_along_data_2 = [haversine(lats[0], longs[0], lats[i], longs[i]) for i in range(len(lats))]
ax.set_xlabel('Along-Track Distance(m)', fontsize = fsize)
ax.set_ylabel('photon height',fontsize = fsize)
ax.set_title('Zoomed-in view of h_ph dataset\nfrom gt1l/heights',fontsize = int(fsize * 1.2 ))
ax.tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))
ax.grid(True)
ax.set_ylim([median_height - buffer_height, median_height + buffer_height])

ax.scatter(np.array(dist_ph_along_data_1), np.array(height_1_buf), s=1, alpha= 0.4, label = "track 1")
ax.scatter(np.array(dist_ph_along_data_2), np.array(heights), s=1, alpha= 0.4, label = "track 2")
ax.legend()
# fig.gca().set_aspect('equal')

![image](https://user-images.githubusercontent.com/26379748/236646205-6a5153e5-5443-4b97-8fa6-3ff492aee1f9.png)
![image](https://user-images.githubusercontent.com/26379748/236677793-ae413a38-d2cc-4a8c-a584-d52f018a803b.png)


In [ ]:
!pip install geopandas
!pip install rasterio

In [ ]:


import rasterio
from rasterio.plot import show

dem_in = rasterio.open('/content/pongdam_dem.tif')
out_data = dem_in.read(1, masked=1)

import numpy as np
import matplotlib.pyplot as plt
#@title **tracks buffer plot with its intersections at centre as well as height vs lat/lon of the buffered region**{ vertical-output: true }
fsize = 10
filename_1 = "/content/processed_ATL03_20190213043533_07140202_005_01.h5" #@param {type:"string"}
filename_2 = "/content/processed_ATL03_20200531180255_10110706_005_01.h5" #@param {type:"string"}
tr_1 = "gt1l" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
tr_2 = "gt1r" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
tr_3 = "gt2l" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
tr_4 = "gt2r" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
tr_5 = "gt3l" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
tr_6 = "gt3r" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]

import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import jax.numpy as jnp
################################################################################
# Calculate distance along track using haversine formula
def haversine(lat1, lon1, lat2, lon2):
  R = 6371.0  # Earth radius in km
  dLat = np.radians(lat2 - lat1)
  dLon = np.radians(lon2 - lon1)
  lat1 = np.radians(lat1)
  lat2 = np.radians(lat2)
  a = np.sin(dLat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dLon/2)**2
  c = 2*np.arcsin(np.sqrt(a))
  return R*c*1000.0
################################################################################
def get_height_lat_lon(filename, track_name):
  # Open the HDF5 file and extract data
  f = h5py.File(filename, 'r')
  h_ph_data = f['/'+str(track_name)+'/heights/h_ph']
  lat_ph_data = f['/'+str(track_name)+'/heights/lat_ph']
  lon_ph_data = f['/'+str(track_name)+'/heights/lon_ph']
  return h_ph_data, lat_ph_data, lon_ph_data
################################################################################
h_ph_data1, lat1, lon1 = get_height_lat_lon(filename_1, tr_6)
h_ph_data2, lat2, lon2 = get_height_lat_lon(filename_2, tr_1)
################################################################################
# Create a new figure and axis
fig, ax = plt.subplots(figsize=(10, 10))
pos = ax.get_position()
w_x, h_y = pos.width, pos.height
################################################################################
lat_1x = np.array(lat1)
lon_1x = np.array(lon1)
lat_2x = np.array(lat2)
lon_2x = np.array(lon2)

# Find the bounding box that covers both tracks with a buffer
min_lon = min(min(lon_1x), min(lon_2x)) - 0.001
max_lon = max(max(lon_1x), max(lon_2x)) + 0.001
min_lat = min(min(lat_1x), min(lat_2x)) - 0.001
max_lat = max(max(lat_1x), max(lat_2x)) + 0.001
M_lon = (max_lon + min_lon)/2
M_lat = (max_lat + min_lat)/2
dx_boundary_x = max(max_lon - min_lon, max_lat - min_lat)
# Set the limits of the axis to the bounding box
ax.set_xlim(M_lon - dx_boundary_x/2, M_lon + dx_boundary_x/2)
ax.set_ylim(M_lat - dx_boundary_x/2, M_lat + dx_boundary_x/2)
show(dem_in, cmap='gist_earth', ax=ax)                            #plotting the dem over which the track points can be plotted

# Add a grid to the plot
ax.grid(True)

# Add a legend to the plot
# Add axis labels and a title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Tracks 1 and 2')
################################################################################
lat_1 = np.array(lat1)
lon_1 = np.array(lon1)
lat_2 = np.array(lat2)
lon_2 = np.array(lon2)
################################################################################
from shapely.geometry import Point
from shapely.ops import unary_union
from shapely.geometry import LineString

# Convert the latitudes and longitudes to shapely Point objects
points_1 = [Point(lon, lat) for lat, lon in zip(lat_1, lon_1)]
points_2 = [Point(lon, lat) for lat, lon in zip(lat_2, lon_2)]

# Find the intersection of the two tracks
intersection = unary_union([LineString(points_1).intersection(LineString(points_2))])
buffer_region = 0.03 #@param {type:"raw"}

# Create a buffer around the intersection point
intersection_buffer = intersection.buffer(buffer_region)
#@title **track in the buffered region with altitude/elevatoion/height buffer control for points visibility with track labels** { vertical-output: true }
# Compute the median height of the tracks within the buffered region
import geopandas as gpd

# Create a GeoDataFrame for the buffered intersection
intersection_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(intersection_buffer))

# Create a mask for selecting points within the buffered region
mask_1 = [intersection_buffer.contains(point) for point in points_1]
mask_2 = [intersection_buffer.contains(point) for point in points_2]

# Select the latitudes, longitudes, and heights within the buffered region for each track
lat_1_buf = lat_1[mask_1]
lon_1_buf = lon_1[mask_1]
height_1_buf = np.array(h_ph_data1)[mask_1]

lat_2_buf = lat_2[mask_2]
lon_2_buf = lon_2[mask_2]
height_2_buf = np.array(h_ph_data2)[mask_2]

##########################################################
# Set the limits of the axis to the bounding box
# Find the bounding box that covers both tracks with a buffer
min_lon = min(min(lon_1_buf), min(lon_2_buf)) - 0.001
max_lon = max(max(lon_1_buf), max(lon_2_buf)) + 0.001
min_lat = min(min(lat_1_buf), min(lat_2_buf)) - 0.001
max_lat = max(max(lat_1_buf), max(lat_2_buf)) + 0.001
M_lon = (max_lon + min_lon)/2
M_lat = (max_lat + min_lat)/2
dx_boundary = max(max_lon - min_lon, max_lat - min_lat)
###########################################################
################################################################################
ax2 = fig.add_axes([0.5 -w_x/2 , 0-h_y , w_x , h_y])
ax2.set_position([pos.x0 , 0-h_y, pos.width, h_y])
#********************************
pos = ax2.get_position()
w_x, h_y = pos.width, pos.height
#********************************

x, y = intersection_buffer.exterior.xy

ax.fill(x, y, alpha=0.6, fc='#fa19b6', ec='#fcc603', linestyle='--', linewidth=1)
ax.scatter(lon_1x, lat_1x, s=1, color='blue', marker='.')
ax.scatter(lon_2x, lat_2x, s=1, color='r', marker='.')
ax.legend(['Track 1', 'Track 2'], loc='lower right')

show(dem_in, cmap='gist_ncar', ax=ax2)                    #showing the dem on which the tracks will be  plotted

ax2.plot(lon_1, lat_1, '.', color='blue', markersize=1)
ax2.plot(lon_2, lat_2, '.', color='red', markersize=1)
ax2.fill(x, y, alpha=0.4, fc='#fa19b6', ec='#03fcc6', linestyle='dotted', linewidth=1)

ax2.set_xlim(M_lon - dx_boundary/2, M_lon + dx_boundary/2)
ax2.set_ylim(M_lat - dx_boundary/2, M_lat + dx_boundary/2)

ax2.grid(True)
ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude')
ax2.set_title('Tracks 1 and 2 with Intersection Buffer')
ax2.legend(['Track 1', 'Track 2'], loc='lower right')
################################################################################< elevation vs latitude
median_height = np.median(np.concatenate((height_1_buf, height_2_buf)))
buffer_height = 3 #@param {type:"raw"}


ax3 = fig.add_axes([0.5 -w_x/2 , 0-h_y , w_x , h_y], sharey=ax2)
ax3.set_position([pos.x0 + pos.width , pos.y0 , w_x/3, pos.height])

ax3.plot(height_1_buf, lat_1_buf, '.', color='blue', markersize=1, label='Track 1')
ax3.plot(height_2_buf, lat_2_buf, '.', color='red', markersize=1, label='Track 2')
ax3.set_xlim(M_lat - dx_boundary/2, M_lat + dx_boundary/2)
ax3.axvspan(median_height - buffer_height, median_height + buffer_height, color='#fdffa3', alpha=0.3)
ax3.yaxis.set_label_position('right')
ax3.yaxis.tick_right()
ax3.set_ylabel('Y-axis label', visible=False)
ax3.set_xlim([median_height - buffer_height, median_height + buffer_height])
ax3.set_xlabel('Height')
ax3.set_ylabel('Latitude')
ax3.set_title('Height vs. Latitude for Tracks 1 and 2 within Buffered Region')
ax3.grid(True,  color='black', alpha = 0.9)
ax3.legend()
################################################################################< elevation vs longitude
ax4 = fig.add_axes([0 , 0 , w_x , h_y/3], sharex=ax2)
ax4.set_position([pos.x0 , pos.y0 - h_y/3 -0.1, pos.width, h_y/3])

ax4.plot(lon_1_buf, height_1_buf, '.', color='blue', markersize=1, label='Track 1')
ax4.plot(lon_2_buf, height_2_buf, '.', color='red', markersize=1, label='Track 2')
ax4.set_xlim(M_lon - dx_boundary/2, M_lon + dx_boundary/2)
ax4.axhspan(median_height - buffer_height, median_height + buffer_height, color='#fdffa3', alpha=0.3)
ax4.set_ylim([median_height - buffer_height, median_height + buffer_height])
ax4.set_xlabel('Longitude')
ax4.set_ylabel('Height')
ax4.set_title('Height vs. Longitude for Tracks 1 and 2 within Buffered Region', loc='center').set_position([.5, 1.01])
ax4.grid(True,  color='black', alpha = 0.9)
ax4.legend()
################################################################################< elevation vs track 1 & 2 distances
ax5 = fig.add_axes([0.5 - w_x/2 , -h_y - h_y/3 -h_y/3 , w_x , h_y/3])
ax5.set_position([pos.x0 ,-h_y - h_y/3 -h_y/3 - 0.1 * 2, pos.width, h_y/3])

dist_ph_along_data_1 = [haversine(lat_1_buf[0], lon_1_buf[0], lat_1_buf[i], lon_1_buf[i]) for i in range(len(lat_1_buf))]

lats = np.flip(lat_2_buf)
longs = np.flip(lon_2_buf)
heights = np.flip(height_2_buf)

dist_ph_along_data_2 = [haversine(lats[0], longs[0], lats[i], longs[i]) for i in range(len(lats))]
ax5.scatter(np.array(dist_ph_along_data_1), np.array(height_1_buf), s=1, alpha= 0.4, label = "track 1")
ax5.scatter(np.array(dist_ph_along_data_2), np.array(heights), s=1, alpha= 0.4, label = "track 2")

ax5.set_ylim([median_height - buffer_height, median_height + buffer_height])
ax5.set_xlabel('Along-Track Distance(m)')
ax5.set_ylabel('photon height')
ax5.set_title('Zoomed-in view of h_ph dataset\nfrom gt1l/heights', loc='center')
ax5.grid(True,  color='black', alpha = 0.9)
ax5.legend()


# Show the plot
plt.subplots_adjust(hspace=0)
plt.show()


In [ ]:
#@title **Elevation & Distance Sample Extraction based on distance threshold in meters** { vertical-output: true }
fsize = 30
filename = "/content/processed_ATL03_20200531180255_10110706_005_01.h5" #@param {type:"string"}
track_name = "gt2l" #@param ["gt1l", "gt1r", "gt2l", "gt2r", "gt3l", "gt3r"]
start_point = 35000 #@param {type:"integer"}
distance_range = 1000 #@param {type:"integer"}
elevation_band_thickness = 500 #@param {type:"integer"}
min_height_zone = elevation_band_thickness

import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import jax.numpy as jnp

def create_geometric_graph(elevations, distances, elevation_weight, distance_weight):
  # Create a complete graph with nodes representing the given points
  num_points = len(elevations)
  edges = [(i, j) for i in range(num_points) for j in range(i+1, num_points)]
  
  # Compute the weights of the edges between all pairs of nodes
  weights = np.zeros((num_points, num_points))
  for i in range(num_points):
      for j in range(i+1, num_points):
          elevation_diff = abs(elevations[i] - elevations[j])
          weights[i][j] = elevation_weight * elevation_diff + distance_weight * abs(distances[i] - distances[j])
          weights[j][i] = weights[i][j]
  
  # Create a GeometricGraph object from the edge list and weights
  geom_graph = (edges, weights)
  
  return geom_graph

# Calculate distance along track using haversine formula
def haversine(lat1, lon1, lat2, lon2):
  R = 6371.0  # Earth radius in km
  dLat = np.radians(lat2 - lat1)
  dLon = np.radians(lon2 - lon1)
  lat1 = np.radians(lat1)
  lat2 = np.radians(lat2)
  a = np.sin(dLat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dLon/2)**2
  c = 2*np.arcsin(np.sqrt(a))
  return R*c*1000.0

w_x = 1
h_y = 1
fig, ax = plt.subplots(figsize=(30,6), dpi = 100)


def extract_elevation_data(filename, track_name, start_point, distance_range, elevation_band_thickness):
  # Open the HDF5 file and extract data
  f = h5py.File(filename, 'r')
  h_ph_data = f['/'+str(track_name)+'/heights/h_ph']
  lat_ph_data = f['/'+str(track_name)+'/heights/lat_ph']
  lon_ph_data = f['/'+str(track_name)+'/heights/lon_ph']

  dist_ph_along_data = [haversine(lat_ph_data[0], lon_ph_data[0], lat_ph_data[i], lon_ph_data[i]) for i in range(len(lat_ph_data))]

  # Select data based on a starting point, distance range, and elevation band thickness
  dist_ph_along_data = np.array(dist_ph_along_data)
  h_ph_data = np.array(h_ph_data)

      # Plot the dataset
  ax.scatter(dist_ph_along_data, h_ph_data, s=1, c = 'y')
  ax.set_xlabel('Along-Track Distance(m)', fontsize = fsize)
  ax.set_ylabel('height of photons',fontsize = fsize)
  ax.set_title('h_ph dataset from gt1l/heights',fontsize = int(fsize * 1.2 ))
  ax.set_ylim(np.median(h_ph_data) - 1500, np.median(h_ph_data) + 1500)
  
  d = get_filtered_indices(dist_ph_along_data, h_ph_data, start_point, distance_range, elevation_band_thickness)
  # Extract elevation and distance data
  elevation = h_ph_data[d]
  distance = dist_ph_along_data[d]
      
  ax.grid(True)
  ax.scatter(np.array(dist_ph_along_data)[d], np.array(h_ph_data)[d], s=2, c = 'g')
  x_mean = np.mean(np.array(dist_ph_along_data)[d])
  y_mean = np.median(np.array(h_ph_data)[d])
  ax.scatter(x_mean, y_mean, s= 40)
  ax.tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))

  rect = patches.Rectangle((x_mean-distance_range/2, y_mean-min_height_zone/2), distance_range, min_height_zone, fill=False, linestyle='dotted', linewidth=2, edgecolor='red')
  ax.add_patch(rect)
  fig.gca().set_aspect('equal')

  ax2 = fig.add_axes([0.25 - w_x/2, -0.4 - h_y/2 , w_x, h_y])
  ax2.set_xlabel('Along-Track Distance(m)', fontsize = fsize)
  ax2.set_ylabel('photon height',fontsize = fsize)
  ax2.set_ylim(y_mean-min_height_zone/2 - 10, y_mean+min_height_zone/2 + 10)
  ax2.set_title('Zoomed-in view of h_ph dataset\nfrom gt1l/heights',fontsize = int(fsize * 1.2 ))
  ax2.tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))
  ax2.grid(True)
  ax2.scatter(np.array(dist_ph_along_data)[d], np.array(h_ph_data)[d], s=20, alpha= 0.4)
  fig.gca().set_aspect('equal')

  from osgeo import gdal

  raster_location = '/content/landsat2/HLSL30.020_B04_doy2019109_aid0001_43N.tif'
  # Load the second raster image into a numpy array using gdal
  crop_ds = gdal.Open(raster_location)
  crop_band = crop_ds.GetRasterBand(1)
  crop_data = crop_band.ReadAsArray()

  # Get the mask of the second band
  crop_mask = crop_band.GetMaskBand().ReadAsArray()

  # Create a masked array from the crop data and mask
  crop_masked = np.ma.masked_where(crop_mask == 0, crop_data)

  # Get the geotransform of the second raster
  crop_gt = crop_ds.GetGeoTransform()

  # Get the minimum and maximum x and y coordinates of the second raster
  crop_xmin = crop_gt[0]
  crop_xmax = crop_gt[0] + (crop_ds.RasterXSize * crop_gt[1])
  crop_ymin = crop_gt[3] + (crop_ds.RasterYSize * crop_gt[5])
  crop_ymax = crop_gt[3]

  import rasterio
  # Open the DEM raster file
  dem_in = rasterio.open('/content/ASTER_DEM.tif')
  dem_data = dem_in.read(1, masked=True)

  extent_data = rasterio.open(raster_location)
  extent = dem_in.window(extent_data.bounds.left,  extent_data.bounds.bottom, extent_data.bounds.right, extent_data.bounds.top)

  out_data = dem_in.read(1, window=extent, masked=True)

  ax3 = fig.add_axes([0.5 - w_x / 2 , 1.4 - h_y/2 , w_x, h_y])

  ax3.set_xlabel('Longitude', fontsize = fsize)
  ax3.set_ylabel('Latitude', fontsize = fsize)
  # ax3.set_ylim(y_mean - min_height_zone/2, y_mean + min_height_zone/2)
  ax3.set_title('Zoomed-in view of h_ph dataset\nfrom gt1l/heights',fontsize = int(fsize * 1.2 ))
  ax3.tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))
  ax3.grid(True)

  ax3.imshow(out_data, extent=[crop_xmin, crop_xmax, crop_ymin, crop_ymax])

  ax3.scatter(lon_ph_data, lat_ph_data, s=1, alpha= 0.4, c ='g')
  ax3.scatter(np.array(lon_ph_data)[d], np.array(lat_ph_data)[d], s=20, alpha= 0.4, c = 'r')
  fig.gca().set_aspect('equal')

  return distance, elevation

# filename = '/content/processed_ATL03_20200211111433_07140602_005_01.h5'
# track_name = 'gt2r'
# start_point = 25500
# distance_range = 300
# elevation_band_thickness = 50

# Create a points array.
def create_points(elevation, distance):
  return jnp.array([
    [elevation[i], distance[i]]
    for i in range(len(elevation))
  ])

def get_filtered_indices(dist_ph_along_data, h_ph_data, start_point, distance_range, elevation_band_thickness):
    c = np.where((dist_ph_along_data > start_point) & (dist_ph_along_data < start_point + distance_range))[0]
    median_height = np.median(h_ph_data[c])
    d = np.where((dist_ph_along_data > start_point) & (dist_ph_along_data < start_point + distance_range) &
                 (h_ph_data > median_height - elevation_band_thickness/2) &
                 (h_ph_data < median_height + elevation_band_thickness/2))[0]
    return d

def create_adjacency_matrix(points, threshold):
  distance_matrix = np.linalg.norm(points[:, None] - points, axis=2)
  mask = distance_matrix < threshold
  adjacency_matrix = np.where(mask, 1, 0)
  return adjacency_matrix

def plot_points_and_connections(elevation, distance, threshold):
  # Create the points array.
  points = create_points(elevation, distance)
  n = len(elevation)
  adjacency_matrix = np.zeros((n, n))
  adjacency_matrix = create_adjacency_matrix(points, threshold)

  # Plot the points and the connections.
  ax4 = fig.add_axes([0.75 -w_x/2 , -0.4 - h_y/2 , w_x, h_y])
  # fig = plt.figure(figsize=(12, 4), dpi=200)
  ax4.set_title('Nearest Neighbour connection view',fontsize = int(fsize * 1.2 ))

  for i in range(n):
    for j in range(n):
      if adjacency_matrix[i, j] == 1:
        ax4.plot( [points[i, 1], points[j, 1]], [points[i, 0], points[j, 0]], 'r-')
  ax4.scatter(points[:, 1], points[:, 0], s = 1)
  fig.gca().set_aspect('equal')
  ax4.tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))
  ax4.grid(True)
  ax4.set_xlabel('distance', fontsize = fsize)
  ax4.set_ylabel('elevation', fontsize = fsize)
  # return fig

distance, elevation = extract_elevation_data(filename, track_name, start_point, distance_range, elevation_band_thickness)
threshold = 3 #@param {type:"integer"}
plot_points_and_connections(elevation, distance, threshold)

raster_file1 = '/content/landsat2/HLSL30.020_B04_doy2019109_aid0001_43N.tif'
raster_file2 = '/content/landsat2/HLSL30.020_B05_doy2019109_aid0001_43N.tif'
raster_file3 = '/content/landsat2/HLSL30.020_B06_doy2019109_aid0001_43N.tif'

# Open the HDF5 file and extract data
f = h5py.File(filename, 'r')
h_ph_data = f['/'+str(track_name)+'/heights/h_ph']
lat_ph_data = f['/'+str(track_name)+'/heights/lat_ph']
lon_ph_data = f['/'+str(track_name)+'/heights/lon_ph']

dist_ph_along_data = [haversine(lat_ph_data[0], lon_ph_data[0], lat_ph_data[i], lon_ph_data[i]) for i in range(len(lat_ph_data))]
d = get_filtered_indices(np.array(dist_ph_along_data), np.array(h_ph_data), start_point, distance_range, elevation_band_thickness)
latitudes = lat_ph_data[d]
longitudes = lon_ph_data[d]
distance = np.array(dist_ph_along_data)[d]
print(len(distance))

def plot_reflectance_by_band(raster_file1, raster_file2, raster_file3, latitudes, longitudes, distance):
  # Load raster file
  ax5 = fig.add_axes([0.5 -w_x/2 , -1.4 - h_y/2 , w_x, h_y])

  print(len(distance), "<<here>>", len(latitudes))
  src1 = rasterio.open(raster_file1)
  src2 = rasterio.open(raster_file2)
  src3 = rasterio.open(raster_file3)

  crs = src1.crs
  transform = src1.transform

  values1 = [x[0] for x in src1.sample(zip(latitudes, longitudes))]
  values2 = [x[0] for x in src2.sample(zip(latitudes, longitudes))]
  values3 = [x[0] for x in src3.sample(zip(latitudes, longitudes))]
  # Normalize the values to be in the range [0, 1]
  values1 = np.interp(values1, (np.min(values1), np.max(values1)), (0, 100))
  values2 = np.interp(values2, (np.min(values2), np.max(values2)), (0, 100))
  values3 = np.interp(values3, (np.min(values3), np.max(values3)), (0, 100))


  ax5.scatter(distance, values1, c='r', s=1, alpha = 0.3)
  ax5.scatter(distance, values2, c='g', s=1, alpha = 0.3)
  ax5.scatter(distance, values3, c='b', s=1, alpha = 0.3)
  fig.gca().set_aspect('equal')
  ax5.tick_params(axis='both', which='major', labelsize=int(fsize * 0.7))
  ax5.grid(True)
  ax5.set_xlabel('distance', fontsize = fsize)
  ax5.set_ylabel('elevation', fontsize = fsize)
  ax5.set_ylim(0, 100)

# plot_reflectance_by_band(raster_file1, raster_file2, raster_file3, latitudes, longitudes, distance)
# plt.show()